<a href="https://colab.research.google.com/github/Sra1panasa/NLP/blob/main/Receipt_FormFilling/concepts_receiptff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Concepts_receipt_FormFilling:**

**Tokenizer**
* A tokenizer is in charge of preparing the inputs for a model. The **library contains tokenizers for all the models**

In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 6.7 MB/s 
     |████████████████████████████████| 3.3 MB 46.8 MB/s 
     |████████████████████████████████| 895 kB 52.8 MB/s 
     |████████████████████████████████| 596 kB 57.7 MB/s 
     |████████████████████████████████| 61 kB 491 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
from transformers import BertTokenizerFast
t = BertTokenizerFast.from_pretrained('bert-base-uncased')
print(t)
t.tokenize("hello,hello, i'm testing")

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


['hello', ',', 'hello', ',', 'i', "'", 'm', 'testing']

* You can also get a mapping of each token to the respecting word and other things:



In [10]:
o = t("hello, i'm testing this efauenufefu", add_special_tokens=False, return_attention_mask=False, return_token_type_ids=False)
print(o)
o.words()

{'input_ids': [7592, 1010, 1045, 1005, 1049, 5604, 2023, 1041, 7011, 24997, 16093, 12879, 2226]}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:336: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  FutureWarning,


[0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 7, 7]

**Understanding word piece,max length,padding:**
* I am working with Text Classification problem where I want to use the BERT model as the base followed by Dense layers. I want to know how does the 3 arguments work? For example, if I have 3 sentences as:

'My name is slim shade and I am an aspiring AI Engineer',

'I am an aspiring AI Engineer',

'My name is Slim'

* SO what will these 3 arguments do? What I think is as follows:

* **max_length=5-** will keep all the sentences as of length 5 strictly
* **padding=max_length-** will add a padding of 1 to the third sentence
* **truncate=True-** will truncate the first and second sentence so that their length will be strictly 5.

is this correct:


**Ans:**

 * What you have assumed is almost correct, however, there are few differences.

* max_length=5, the max_length specifies the length of the tokenized text. By default, BERT performs word-piece tokenization. For example the word "playing" can be split into "play" and "##ing" (This may not be very precise, but just to help you understand about word-piece tokenization), followed by adding [CLS] token at the beginning of the sentence, and [SEP] token at the end of sentence. Thus, it first tokenizes the sentence, truncates it to max_length-2 (if truncation=True), then prepend [CLS] at the beginning and [SEP] token at the end.(So a total length of max_length)

* padding='max_length', In this example it is not very evident that the 3rd example will be padded, as the length exceeds 5 after appending [CLS] and [SEP] tokens. However, if you have a max_length of 10. The tokenized text corresponds to [101, 2026, 2171, 2003, 11754, 102, 0, 0, 0, 0], where 101 is id of [CLS] and 102 is id of [SEP] tokens. Thus, padded by zeros to make all the text to the length of max_length

* Likewise, truncate=True will ensure that the max_length is strictly adhered, i.e, longer sentences are truncated to max_length only if truncate=True

In [2]:
from transformers import BertTokenizerFast
import torch
text=['love','ai']
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokens = tokenizer.batch_encode_plus(text,max_length=2,padding='max_length', truncation=True)
print(tokens)
  
text_seq = torch.tensor(tokens['input_ids'])
text_mask = torch.tensor(tokens['attention_mask'])
print(text_seq)
print(text_mask)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': [[101, 102], [101, 102]], 'token_type_ids': [[0, 0], [0, 0]], 'attention_mask': [[1, 1], [1, 1]]}
tensor([[101, 102],
        [101, 102]])
tensor([[1, 1],
        [1, 1]])


**Trainer and training arguments:**
* We can then fine-tune the model with Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    save_total_limit=1,              # limit the total amount of checkpoints. Deletes the older checkpoints.    
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()
